In [1]:
# 任务8.2 分析财政收入数据特征的相关性

# 代码 8-2
import numpy as np
import pandas as pd
inputfile = 'data.csv' ## 输入的数据文件
data = pd.read_csv(inputfile) ## 读取数据
## 保留两位小数
print('相关系数矩阵为：',np.round(data.corr(method = 'pearson'), 2))


相关系数矩阵为：        x1    x2    x3    x4    x5    x6    x7    x8    x9   x10   x11   x12  \
x1   1.00  0.95  0.95  0.97  0.97  0.99  0.95  0.97  0.98  0.98 -0.29  0.94   
x2   0.95  1.00  1.00  0.99  0.99  0.92  0.99  0.99  0.98  0.98 -0.13  0.89   
x3   0.95  1.00  1.00  0.99  0.99  0.92  1.00  0.99  0.98  0.99 -0.15  0.89   
x4   0.97  0.99  0.99  1.00  1.00  0.95  0.99  1.00  0.99  1.00 -0.19  0.91   
x5   0.97  0.99  0.99  1.00  1.00  0.95  0.99  1.00  0.99  1.00 -0.18  0.90   
x6   0.99  0.92  0.92  0.95  0.95  1.00  0.93  0.95  0.97  0.96 -0.34  0.95   
x7   0.95  0.99  1.00  0.99  0.99  0.93  1.00  0.99  0.98  0.99 -0.15  0.89   
x8   0.97  0.99  0.99  1.00  1.00  0.95  0.99  1.00  0.99  1.00 -0.15  0.90   
x9   0.98  0.98  0.98  0.99  0.99  0.97  0.98  0.99  1.00  0.99 -0.23  0.91   
x10  0.98  0.98  0.99  1.00  1.00  0.96  0.99  1.00  0.99  1.00 -0.17  0.90   
x11 -0.29 -0.13 -0.15 -0.19 -0.18 -0.34 -0.15 -0.15 -0.23 -0.17  1.00 -0.43   
x12  0.94  0.89  0.89  0.91  0.90  0.95  0.

In [2]:
# 任务8.3 使用Lasso回归选取财政收入预测的关键特征

# 代码 8-3
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
inputfile = 'data.csv' #输入的数据文件
data = pd.read_csv(inputfile) #读取数据
lasso = Lasso(1000)  #调用Lasso()函数，设置λ的值为1000
lasso.fit(data.iloc[:,0:13],data['y'])
print('相关系数为：',np.round(lasso.coef_,5))  #输出结果，保留五位小数

## 计算相关系数非零的个数
print('相关系数非零个数为：',np.sum(lasso.coef_ != 0))

mask = lasso.coef_ != 0  #返回一个相关系数是否为零的布尔数组
print('相关系数是否为零：',mask)

outputfile = 'new_reg_data.csv'  #输出的数据文件
new_reg_data = data.iloc[:, mask]  #返回相关系数非零的数据
new_reg_data.to_csv(outputfile)  #存储数据
print('输出数据的维度为：',new_reg_data.shape)  #查看输出数据的维度


相关系数为： [-1.8000e-04 -0.0000e+00  1.2414e-01 -1.0310e-02  6.5400e-02  1.2000e-04
  3.1741e-01  3.4900e-02 -0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 -4.0300e-02]
相关系数非零个数为： 8
相关系数是否为零： [ True False  True  True  True  True  True  True False False False False
  True]
输出数据的维度为： (20, 8)


D:\Python\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [4]:
# 任务8.4 使用灰色预测和SVR构建财政收入预测模型

# 代码 8-4
import numpy as np
import pandas as pd
from GM11 import GM11 ## 引入自编的灰色预测函数
inputfile = 'new_reg_data.csv' ##输入的数据文件
inputfile1 = 'data.csv' ## 输入的数据文件
new_reg_data = pd.read_csv(inputfile) ## 读取经过特征选择后的数据
data = pd.read_csv(inputfile1) ##读取总的数据
new_reg_data.index = range(1994, 2014)
new_reg_data.loc[2014] = None
new_reg_data.loc[2015] = None
l = ['x1', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x13']
for i in l:
    f = GM11(new_reg_data.loc[range(1994, 2014),i].values)[0]
    new_reg_data.loc[2014,i] = f(len(new_reg_data)-1)#2014年预测结果
    new_reg_data.loc[2015,i] = f(len(new_reg_data)) ##2015年预测结果
    new_reg_data[i] = new_reg_data[i].round(2) ## 保留两位小数
outputfile = 'new_reg_data_GM11.xls' ## 灰色预测后保存的路径
y = list(data['y'].values) ## 提取财政收入列，合并至新数据框中
y.extend([np.nan,np.nan])
new_reg_data['y'] = y
new_reg_data.to_excel(outputfile) ## 结果输出
print('预测结果为：',new_reg_data.loc[2014:2015,:]) ##预测结果展示

# 代码 8-5
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVR
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score,\
mean_absolute_error,mean_squared_error,\
median_absolute_error,r2_score
inputfile = 'new_reg_data_GM11.xls' #灰色预测后保存的路径
data = pd.read_excel(inputfile) #读取数据
feature = ['x1', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x13'] 
data_train = data.loc[range(1994,2014)].copy()#取2014年前的数据建模
data_mean = data_train.mean()
data_std = data_train.std()
data_train = (data_train - data_mean)/data_std #数据标准化
x_train = data_train[feature].values #特征数据
y_train = data_train['y'].values #标签数据
linearsvr = LinearSVR()   #调用LinearSVR()函数
linearsvr.fit(x_train,y_train)
x = ((data[feature] - data_mean[feature])/ \
data_std[feature]).values  #预测，并还原结果。
data[u'y_pred'] = linearsvr.predict(x) * \
data_std['y'] + data_mean['y']
## SVR预测后保存的结果
outputfile = 'new_reg_data_GM11_revenue.xls'
data.to_excel(outputfile)
print('真实值与预测值分别为：',data[['y','y_pred']])

print('预测图为：')
data[['y','y_pred']].plot(subplots = True,style=['b-o','r-*'])
plt.show()


预测结果为：       Unnamed: 0          x1       x3        x4        x5          x6  \
2014         NaN  8142148.24  7042.31  43611.84  35046.63  8505522.58   
2015         NaN  8460489.28  8166.92  47792.22  38384.22  8627139.31   

           x7        x8       x13   y  
2014  4600.40  18686.28  44506.47 NaN  
2015  5214.78  21474.47  49945.88 NaN  


KeyError: "None of [Int64Index([1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,\n            2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013],\n           dtype='int64')] are in the [index]"